# Causal Forest Difference-in-Differences Analysis
## Part 1: Data, Methodology, and Core Implementation

### Singapore's Prime Location Housing (PLH) Reform Evaluation

---

**Author:** Naia  Nathan
**Affiliation:** Columbia University, Economics  
**Advisor:** Professor Jonathan Dingel  

---

## Abstract

This notebook implements the **Causal Forest Difference-in-Differences (CF-DiD)** methodology to evaluate Singapore's Prime Location Housing (PLH) reform, introduced in November 2021. The reform aimed to reduce speculative demand for public housing in prime locations by extending minimum occupancy periods, introducing subsidy clawbacks, and restricting rental options.

The analysis follows the methodological framework of:

> **Gavrilova, E., Langørgen, A., & Zoutman, F. T. (2025). "Difference-in-Difference Causal Forests With an Application to Payroll Tax Incidence in Norway." *Journal of Applied Econometrics*.**

---

## Research Design Summary

| Element | Specification |
|---------|---------------|
| **Treatment** | Prime Location Housing (PLH) designation (Nov 2021) |
| **Treated Group** | `everprime = 1` (locations ever designated as prime) |
| **Control Group** | `everprime = 0` (non-prime locations) |
| **Outcome** | `log_prob_first` (log probability of first-timer allocation success) |
| **Panel Unit** | Town × Room type combinations |
| **Pre-Policy Period** | November 2011 – October 2021 |
| **Post-Policy Period** | November 2021 – August 2023 |

---

## Notebook Structure

1. **Setup & Configuration**
2. **Data Loading & Preprocessing**
3. **Feature Engineering & VIF Analysis**
4. **CF-DiD Methodology & Implementation**
5. **Core Model Estimation**

Results, validation tests, and figures are in **Part 2**.

---
## 1. Setup & Configuration

In [1]:
"""
================================================================================
SECTION 1: SETUP AND CONFIGURATION
================================================================================
This section loads all required libraries and configures paths.

Requirements:
- pandas, numpy, scipy, sklearn, statsmodels, matplotlib, seaborn
- Data file: 'btodata_complete.xlsx' in DATA_OUTPUT directory
"""

# Core libraries
import pandas as pd
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Machine learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import cross_val_score

# Econometrics
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Visualization (colorblind-friendly Okabe-Ito palette)
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

# Paths
from pathlib import Path
import sys

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries loaded successfully.")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")

Libraries loaded successfully.
NumPy: 2.0.2
Pandas: 2.2.2


In [2]:
# =============================================================================
# PATH CONFIGURATION
# =============================================================================
# Modify these paths according to your environment

# Option 1: Google Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
    THESIS_ROOT = Path("/content/drive/MyDrive/thesis")
    ENVIRONMENT = "colab"
except ImportError:
    # Option 2: Local environment
    THESIS_ROOT = Path("./")  # Adjust to your local path
    ENVIRONMENT = "local"

sys.path.insert(0, str(THESIS_ROOT))

# Try importing config, fall back to defaults
try:
    from config import RAW_DATA, CLEAN_DATA, DATA_OUTPUT
except ImportError:
    RAW_DATA = THESIS_ROOT / "rawdata"
    CLEAN_DATA = THESIS_ROOT / "cleandata"
    DATA_OUTPUT = THESIS_ROOT / "data_output"

# Output directory
OUTPUT = THESIS_ROOT / "FINAL_OUTPUT"
OUTPUT.mkdir(parents=True, exist_ok=True)

print(f"Environment: {ENVIRONMENT}")
print(f"Data source: {DATA_OUTPUT}")
print(f"Output directory: {OUTPUT}")

Mounted at /content/drive
Environment: colab
Data source: /content/drive/MyDrive/thesis/data_output
Output directory: /content/drive/MyDrive/thesis/FINAL_OUTPUT


In [3]:
# =============================================================================
# VISUALIZATION CONFIGURATION
# =============================================================================
# Colorblind-friendly palette (Okabe-Ito)
# Reference: https://jfly.uni-koeln.de/color/

CB_COLORS = {
    'blue': '#0072B2',       # Main blue
    'orange': '#E69F00',     # Main orange
    'green': '#009E73',      # Teal green
    'pink': '#CC79A7',       # Reddish purple
    'lightblue': '#56B4E9',  # Sky blue
    'yellow': '#F0E442',     # Yellow
    'vermillion': '#D55E00', # Vermillion (for emphasis)
    'black': '#000000',
    'gray': '#999999'
}

# LaTeX-compatible styling
plt.rcParams.update({
    'font.family': 'serif',
    'font.serif': ['Computer Modern Roman', 'Times New Roman', 'DejaVu Serif'],
    'text.usetex': False,  # Set True if LaTeX is installed
    'font.size': 11,
    'axes.labelsize': 12,
    'axes.titlesize': 13,
    'legend.fontsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'figure.figsize': (10, 6),
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.spines.top': False,
    'axes.spines.right': False
})

print("Visualization styling configured (colorblind-friendly Okabe-Ito palette).")

Visualization styling configured (colorblind-friendly Okabe-Ito palette).


---
## 2. Data Loading & Preprocessing

The dataset contains Build-to-Order (BTO) housing allocation outcomes from Singapore's Housing Development Board (HDB), spanning November 2011 to August 2023. Each observation represents a town × room type × allocation cycle combination.

In [4]:
# =============================================================================
# DATA LOADING
# =============================================================================

# Load main dataset
df = pd.read_excel(DATA_OUTPUT / 'btodata_complete.xlsx')

# Create datetime column
df['date'] = pd.to_datetime(df['year_month'])

# Create panel identifier: town × room type
df['town_room_id'] = df['town'].astype(str) + '_' + df['room'].astype(str)

# Create numeric panel ID for fixed effects
le = LabelEncoder()
df['panel_id'] = le.fit_transform(df['town_room_id'])

# Create time period index (months from start)
df = df.sort_values('date')
df['time_period'] = ((df['date'] - df['date'].min()).dt.days / 30).astype(int)

# Treatment date and period
TREATMENT_DATE = pd.Timestamp('2021-11-01')
TREATMENT_PERIOD = int(((TREATMENT_DATE - df['date'].min()).days / 30))

# Sort data
df = df.sort_values(['town_room_id', 'date']).reset_index(drop=True)

print("="*60)
print("DATA SUMMARY")
print("="*60)
print(f"Total observations: {len(df):,}")
print(f"Unique panel units (town × room): {df['town_room_id'].nunique()}")
print(f"Date range: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
print(f"Treatment date: {TREATMENT_DATE.strftime('%Y-%m-%d')} (period {TREATMENT_PERIOD})")
print("\nTreatment status distribution:")
print(df.groupby('everprime').agg({
    'town_room_id': 'nunique',
    'log_prob_first': 'mean'
}).rename(columns={'town_room_id': 'n_panels', 'log_prob_first': 'mean_outcome'}))

DATA SUMMARY
Total observations: 596
Unique panel units (town × room): 73
Date range: 2011-11-01 to 2025-10-01
Treatment date: 2021-11-01 (period 121)

Treatment status distribution:
           n_panels  mean_outcome
everprime                        
0                57     -0.693370
1                16     -0.908915


In [5]:
# =============================================================================
# DEFINE ANALYSIS PERIOD
# =============================================================================

# Filter to analysis period (end at Aug 2023 to ensure sufficient post-treatment data)
analysis_df = df[df['date'] <= '2023-08-31'].copy()

# Create post-treatment indicator
analysis_df['post'] = (analysis_df['date'] >= '2021-11-01').astype(int)

# Split data
pre_policy = analysis_df[analysis_df['post'] == 0]
post_policy = analysis_df[analysis_df['post'] == 1]

print("\n" + "="*60)
print("ANALYSIS PERIOD BREAKDOWN")
print("="*60)
print(f"Pre-policy observations (before Nov 2021): {len(pre_policy):,}")
print(f"Post-policy observations (Nov 2021 - Aug 2023): {len(post_policy):,}")
print(f"Total analysis observations: {len(analysis_df):,}")

# Treatment-control breakdown
breakdown = analysis_df.groupby(['everprime', 'post']).size().unstack(fill_value=0)
breakdown.columns = ['Pre-Treatment', 'Post-Treatment']
breakdown.index = ['Control (Non-Prime)', 'Treated (Prime)']
print("\nObservation counts by group and period:")
print(breakdown)


ANALYSIS PERIOD BREAKDOWN
Pre-policy observations (before Nov 2021): 417
Post-policy observations (Nov 2021 - Aug 2023): 76
Total analysis observations: 493

Observation counts by group and period:
                     Pre-Treatment  Post-Treatment
Control (Non-Prime)            367              53
Treated (Prime)                 50              23


---
## 3. Feature Engineering & VIF Analysis

We conduct Variance Inflation Factor (VIF) analysis to identify and remove multicollinear features. This is critical for the random forest's ability to properly attribute importance across correlated features.

In [6]:
# =============================================================================
# FEATURE DEFINITIONS
# =============================================================================

# Continuous features for the model
CONTINUOUS_FEATURES = [
    # Distance features (spatial characteristics)
    'dist_park_km', 'dist_supermarket_km', 'dist_hawker_km', 'dist_mrt_km',
    'dist_road_km', 'dist_hdb_existing_km', 'dist_hdb_uc_km', 'dist_mbfc_km',
    'dist_mall_km', 'dist_elite_km', 'LON',

    # Unit characteristics
    'room_num', 'wait_time_month',

    # Macroeconomic variables
    'cpi', 'sti_close', 'sti_volume', 'pc_gni', 'pc_gdp', 'ave_sora_mth',

    # Cycle characteristics
    'cycle_total_supply', 'cycle_median_dist_mbfc',
    'next_cycle_total_supply', 'next_cycle_median_dist_mbfc'
]

CATEGORICAL_FEATURES = ['town', 'quarter']
BINARY_FEATURES = ['mature', 'covid']

print(f"Initial feature count: {len(CONTINUOUS_FEATURES)} continuous features")

Initial feature count: 23 continuous features


In [7]:
# =============================================================================
# VIF ANALYSIS FUNCTIONS
# =============================================================================

def calculate_vif(df, features):
    """
    Calculate Variance Inflation Factor for each feature.

    VIF measures multicollinearity:
    - VIF = 1: No correlation with other features
    - VIF > 5: Moderate multicollinearity (consider removal)
    - VIF > 10: High multicollinearity (strong case for removal)

    Parameters:
    -----------
    df : DataFrame
        Data containing the features
    features : list
        List of feature column names

    Returns:
    --------
    DataFrame with features and their VIF values, sorted descending
    """
    X = df[features].dropna()
    X = StandardScaler().fit_transform(X)
    X = pd.DataFrame(X, columns=features)

    vif_data = pd.DataFrame()
    vif_data['Feature'] = features
    vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return vif_data.sort_values('VIF', ascending=False)


def iterative_vif_selection(df, features, threshold=5):
    """
    Iteratively remove features with highest VIF until all are below threshold.

    Parameters:
    -----------
    df : DataFrame
        Data containing the features
    features : list
        Initial list of feature column names
    threshold : float
        VIF threshold (default 5)

    Returns:
    --------
    tuple: (remaining_features, removed_features, final_vif_df)
    """
    remaining = features.copy()
    removed = []

    while True:
        vif = calculate_vif(df, remaining)
        max_vif = vif['VIF'].max()

        if max_vif < threshold:
            break

        # Remove feature with highest VIF
        feature_to_remove = vif.iloc[0]['Feature']
        remaining.remove(feature_to_remove)
        removed.append((feature_to_remove, max_vif))

    return remaining, removed, calculate_vif(df, remaining)

In [8]:
# =============================================================================
# RUN VIF ANALYSIS
# =============================================================================

# Initial VIF analysis
print("="*60)
print("INITIAL VIF ANALYSIS")
print("="*60)
vif_initial = calculate_vif(analysis_df, CONTINUOUS_FEATURES)
print(vif_initial.to_string(index=False))

# Iterative selection with threshold=5
print("\n" + "="*60)
print("ITERATIVE VIF SELECTION (threshold=5)")
print("="*60)

final_features, removed_features, final_vif = iterative_vif_selection(
    analysis_df, CONTINUOUS_FEATURES, threshold=5
)

print("\nFeatures removed due to high VIF:")
for feat, vif_val in removed_features:
    print(f"  - {feat}: VIF = {vif_val:.2f}")

print(f"\nFinal selected features ({len(final_features)}):")
print(final_vif.to_string(index=False))

INITIAL VIF ANALYSIS
                    Feature        VIF
                     pc_gdp 147.218811
                     pc_gni 129.539884
                        cpi   9.674119
               ave_sora_mth   4.334480
               dist_mbfc_km   4.285315
               dist_road_km   4.091847
              dist_elite_km   3.983463
       dist_hdb_existing_km   3.361470
                 sti_volume   3.023693
        dist_supermarket_km   2.666828
                dist_mrt_km   2.278741
               dist_park_km   2.144368
                        LON   2.123123
            wait_time_month   1.997198
                  sti_close   1.928975
    next_cycle_total_supply   1.807315
         cycle_total_supply   1.746267
             dist_hawker_km   1.527340
next_cycle_median_dist_mbfc   1.413499
     cycle_median_dist_mbfc   1.406789
             dist_hdb_uc_km   1.405424
               dist_mall_km   1.292405
                   room_num   1.050599

ITERATIVE VIF SELECTION (threshold=5)

Fea

In [9]:
# =============================================================================
# FINALIZE ANALYSIS FEATURES
# =============================================================================

# Use VIF-selected features
ANALYSIS_FEATURES = final_features.copy()

# Clean data: drop rows with missing values in analysis features
analysis_df_clean = analysis_df.dropna(subset=ANALYSIS_FEATURES + ['log_prob_first']).copy()

print("="*60)
print("FINAL ANALYSIS DATASET")
print("="*60)
print(f"Features used: {len(ANALYSIS_FEATURES)}")
print(f"Observations after cleaning: {len(analysis_df_clean):,}")
print(f"Treated post-reform observations: {len(analysis_df_clean[(analysis_df_clean['everprime']==1) & (analysis_df_clean['post']==1)])}")

# Save VIF results
vif_results = pd.DataFrame({
    'Feature': [f[0] for f in removed_features] + final_features,
    'Status': ['Removed']*len(removed_features) + ['Retained']*len(final_features),
    'Final_VIF': [f[1] for f in removed_features] + final_vif['VIF'].tolist()
})
vif_results.to_csv(OUTPUT / 'vif_analysis.csv', index=False)
print(f"\nVIF results saved to: {OUTPUT / 'vif_analysis.csv'}")

FINAL ANALYSIS DATASET
Features used: 21
Observations after cleaning: 493
Treated post-reform observations: 23

VIF results saved to: /content/drive/MyDrive/thesis/FINAL_OUTPUT/vif_analysis.csv


---
## 4. CF-DiD Methodology & Implementation

### Theoretical Foundation

The Causal Forest Difference-in-Differences (CF-DiD) methodology, following **Gavrilova, Langørgen, & Zoutman (2025)**, addresses the challenge of estimating causal effects when:

1. **Standard parallel trends fails** due to heterogeneity in pre-treatment trajectories
2. **Treatment groups are sparse** (few treated units)
3. **High-dimensional covariates** need to be flexibly controlled

### Key Assumptions

1. **Conditional Parallel Trends**: After conditioning on observables $X$, treated and control groups would have followed parallel paths absent treatment.

$$E[Y_{it}(0) - Y_{it-1}(0) | D_i=1, X_i] = E[Y_{it}(0) - Y_{it-1}(0) | D_i=0, X_i]$$

2. **No Anticipation**: Treatment effects only begin after the policy implementation date.

3. **SUTVA**: No spillovers between treated and control units.

### Estimation Procedure

**Step 1**: Train a random forest on control group data to learn $\hat{\mu}(X, t) = E[Y | X, t, D=0]$

**Step 2**: Predict counterfactual outcomes for treated units: $\hat{Y}_{it}(0) = \hat{\mu}(X_i, t)$

**Step 3**: Compute individual treatment effects: $\hat{\tau}_i = Y_{it} - \hat{Y}_{it}(0)$

**Step 4**: Average Treatment Effect on the Treated (ATT): $\hat{\tau}_{ATT} = \frac{1}{N_T} \sum_{i: D_i=1, t>\tau} \hat{\tau}_i$

In [10]:
# =============================================================================
# CF-DiD ESTIMATOR CLASS
# =============================================================================

class CausalForestDiD:
    """
    Causal Forest Difference-in-Differences Estimator

    Implements the methodology from:
    Gavrilova, E., Langørgen, A., & Zoutman, F. T. (2025).
    "Difference-in-Difference Causal Forests With an Application to
    Payroll Tax Incidence in Norway." Journal of Applied Econometrics.

    The estimator:
    1. Trains a random forest on control units to predict outcomes
    2. Uses the trained model to generate counterfactual predictions for treated units
    3. Computes treatment effects as the difference between observed and counterfactual
    4. Provides inference via clustered bootstrap

    Parameters:
    -----------
    n_trees : int
        Number of trees in the random forest (default: 500)
    max_depth : int or None
        Maximum depth of trees (default: None, unlimited)
    min_samples_leaf : int
        Minimum samples per leaf node (default: 5)
    n_bootstrap : int
        Number of bootstrap iterations for inference (default: 200)
    """

    def __init__(self, n_trees=500, max_depth=None, min_samples_leaf=5, n_bootstrap=200):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.n_bootstrap = n_bootstrap
        self.outcome_model = None
        self.att_estimate = None
        self.att_se = None
        self.att_ci_lower = None
        self.att_ci_upper = None

    def _prepare_features(self, df, feature_cols):
        """Prepare feature matrix."""
        return df[feature_cols].values

    def fit(self, df, outcome_col, treatment_col, post_col, panel_col, time_col, feature_cols):
        """
        Fit the CF-DiD model.

        Parameters:
        -----------
        df : DataFrame
            Panel dataset
        outcome_col : str
            Name of outcome variable
        treatment_col : str
            Name of treatment indicator (0/1, time-invariant)
        post_col : str
            Name of post-treatment period indicator
        panel_col : str
            Name of panel unit identifier
        time_col : str
            Name of time period variable
        feature_cols : list
            List of feature column names
        """
        # Store column names
        self.outcome_col = outcome_col
        self.treatment_col = treatment_col
        self.post_col = post_col
        self.panel_col = panel_col
        self.time_col = time_col
        self.feature_cols = feature_cols

        # =====================================================================
        # STEP 1: Train outcome model on control group (all periods)
        # =====================================================================
        control_data = df[df[treatment_col] == 0].copy()

        X_control = self._prepare_features(control_data, feature_cols)
        y_control = control_data[outcome_col].values

        # Add time fixed effects via dummies
        time_dummies = pd.get_dummies(control_data[time_col], prefix='t')
        X_control_full = np.hstack([X_control, time_dummies.values])
        self.time_dummy_cols = time_dummies.columns.tolist()

        # Train random forest
        self.outcome_model = RandomForestRegressor(
            n_estimators=self.n_trees,
            max_depth=self.max_depth,
            min_samples_leaf=self.min_samples_leaf,
            random_state=42,
            n_jobs=-1
        )
        self.outcome_model.fit(X_control_full, y_control)

        # =====================================================================
        # STEP 2: Predict counterfactual for treated units in post-period
        # =====================================================================
        treated_post = df[(df[treatment_col] == 1) & (df[post_col] == 1)].copy()

        X_treated = self._prepare_features(treated_post, feature_cols)
        time_dummies_treated = pd.get_dummies(treated_post[time_col], prefix='t')

        # Align time dummies with training set
        for col in self.time_dummy_cols:
            if col not in time_dummies_treated.columns:
                time_dummies_treated[col] = 0
        time_dummies_treated = time_dummies_treated[self.time_dummy_cols]

        X_treated_full = np.hstack([X_treated, time_dummies_treated.values])

        # Generate counterfactual predictions
        y_counterfactual = self.outcome_model.predict(X_treated_full)
        y_observed = treated_post[outcome_col].values

        # =====================================================================
        # STEP 3: Compute individual treatment effects
        # =====================================================================
        self.individual_effects = y_observed - y_counterfactual
        self.treated_post_df = treated_post.copy()
        self.treated_post_df['tau_hat'] = self.individual_effects
        self.treated_post_df['y_counterfactual'] = y_counterfactual

        # =====================================================================
        # STEP 4: Compute ATT and inference
        # =====================================================================
        self.att_estimate = np.mean(self.individual_effects)
        self._bootstrap_inference(df)

        return self

    def _bootstrap_inference(self, df):
        """
        Clustered bootstrap for ATT standard error.

        Clusters at the panel level to account for within-panel correlation.
        """
        bootstrap_atts = []

        for b in range(self.n_bootstrap):
            # Resample panels (clustered bootstrap)
            panels = df[self.panel_col].unique()
            sampled_panels = np.random.choice(panels, size=len(panels), replace=True)

            # Create bootstrap sample
            boot_dfs = []
            for i, p in enumerate(sampled_panels):
                panel_df = df[df[self.panel_col] == p].copy()
                panel_df['boot_panel'] = i
                boot_dfs.append(panel_df)
            boot_df = pd.concat(boot_dfs, ignore_index=True)

            # Re-estimate on bootstrap sample
            control_data = boot_df[boot_df[self.treatment_col] == 0]
            X_control = self._prepare_features(control_data, self.feature_cols)
            y_control = control_data[self.outcome_col].values

            time_dummies = pd.get_dummies(control_data[self.time_col], prefix='t')
            for col in self.time_dummy_cols:
                if col not in time_dummies.columns:
                    time_dummies[col] = 0
            time_dummies = time_dummies.reindex(columns=self.time_dummy_cols, fill_value=0)
            X_control_full = np.hstack([X_control, time_dummies.values])

            # Fit model (fewer trees for speed)
            rf = RandomForestRegressor(
                n_estimators=100,
                max_depth=self.max_depth,
                min_samples_leaf=self.min_samples_leaf,
                random_state=42 + b,
                n_jobs=-1
            )
            rf.fit(X_control_full, y_control)

            # Predict counterfactual
            treated_post = boot_df[(boot_df[self.treatment_col] == 1) &
                                   (boot_df[self.post_col] == 1)]
            if len(treated_post) == 0:
                continue

            X_treated = self._prepare_features(treated_post, self.feature_cols)
            time_dummies_treated = pd.get_dummies(treated_post[self.time_col], prefix='t')
            for col in self.time_dummy_cols:
                if col not in time_dummies_treated.columns:
                    time_dummies_treated[col] = 0
            time_dummies_treated = time_dummies_treated.reindex(
                columns=self.time_dummy_cols, fill_value=0
            )
            X_treated_full = np.hstack([X_treated, time_dummies_treated.values])

            y_cf = rf.predict(X_treated_full)
            y_obs = treated_post[self.outcome_col].values

            boot_att = np.mean(y_obs - y_cf)
            bootstrap_atts.append(boot_att)

        self.bootstrap_atts = np.array(bootstrap_atts)
        self.att_se = np.std(bootstrap_atts)
        self.att_ci_lower = np.percentile(bootstrap_atts, 2.5)
        self.att_ci_upper = np.percentile(bootstrap_atts, 97.5)

    def summary(self):
        """
        Print and return summary of estimation results.
        """
        t_stat = self.att_estimate / self.att_se if self.att_se > 0 else np.nan
        p_value = 2 * (1 - stats.norm.cdf(abs(t_stat)))

        print("="*60)
        print("CF-DiD ESTIMATION RESULTS")
        print("="*60)
        print(f"\nAverage Treatment Effect on the Treated (ATT):")
        print(f"  Estimate:     {self.att_estimate:.4f}")
        print(f"  Std. Error:   {self.att_se:.4f}")
        print(f"  t-statistic:  {t_stat:.4f}")
        print(f"  p-value:      {p_value:.4f}")
        print(f"  95% CI:       [{self.att_ci_lower:.4f}, {self.att_ci_upper:.4f}]")
        print(f"\nNumber of treated post-treatment obs: {len(self.individual_effects)}")
        print(f"Bootstrap iterations: {self.n_bootstrap}")
        print("="*60)

        return {
            'ATT': self.att_estimate,
            'SE': self.att_se,
            't_stat': t_stat,
            'p_value': p_value,
            'CI_lower': self.att_ci_lower,
            'CI_upper': self.att_ci_upper,
            'n_treated_post': len(self.individual_effects)
        }

print("CausalForestDiD class defined successfully.")

CausalForestDiD class defined successfully.


---
## 5. Core Model Estimation

We now estimate the main CF-DiD model. The key result is the Average Treatment Effect on the Treated (ATT), which measures the impact of PLH reform on first-timer allocation probability for prime locations.

In [11]:
# =============================================================================
# FIT CF-DiD MODEL
# =============================================================================

print("Fitting CF-DiD model (this may take a few minutes)...\n")

# Initialize and fit model
cfdid = CausalForestDiD(
    n_trees=500,
    min_samples_leaf=5,
    n_bootstrap=200
)

cfdid.fit(
    df=analysis_df_clean,
    outcome_col='log_prob_first',
    treatment_col='everprime',
    post_col='post',
    panel_col='panel_id',
    time_col='time_period',
    feature_cols=ANALYSIS_FEATURES
)

# Print summary
att_results = cfdid.summary()

Fitting CF-DiD model (this may take a few minutes)...

CF-DiD ESTIMATION RESULTS

Average Treatment Effect on the Treated (ATT):
  Estimate:     0.4987
  Std. Error:   0.2286
  t-statistic:  2.1812
  p-value:      0.0292
  95% CI:       [0.1901, 1.0120]

Number of treated post-treatment obs: 23
Bootstrap iterations: 200


In [12]:
# =============================================================================
# SAVE MAIN RESULTS
# =============================================================================

# Create results table
att_table = pd.DataFrame({
    'Statistic': ['ATT Estimate', 'Standard Error', 't-statistic', 'p-value',
                  '95% CI Lower', '95% CI Upper', 'N (treated post)'],
    'Value': [
        f"{att_results['ATT']:.4f}",
        f"{att_results['SE']:.4f}",
        f"{att_results['t_stat']:.4f}",
        f"{att_results['p_value']:.4f}",
        f"{att_results['CI_lower']:.4f}",
        f"{att_results['CI_upper']:.4f}",
        str(att_results['n_treated_post'])
    ]
})

att_table.to_csv(OUTPUT / 'cfdid_att_results.csv', index=False)
print("Main results saved to: cfdid_att_results.csv")
print("\n" + att_table.to_string(index=False))

Main results saved to: cfdid_att_results.csv

       Statistic  Value
    ATT Estimate 0.4987
  Standard Error 0.2286
     t-statistic 2.1812
         p-value 0.0292
    95% CI Lower 0.1901
    95% CI Upper 1.0120
N (treated post)     23


In [13]:
# =============================================================================
# INTERPRETATION OF RESULTS
# =============================================================================

# Convert log effect to percentage change
pct_change = (np.exp(att_results['ATT']) - 1) * 100
pct_lower = (np.exp(att_results['CI_lower']) - 1) * 100
pct_upper = (np.exp(att_results['CI_upper']) - 1) * 100

print("="*60)
print("INTERPRETATION")
print("="*60)
print(f"\nThe PLH reform is estimated to increase first-timer allocation")
print(f"probability in prime locations by approximately {pct_change:.1f}%")
print(f"(95% CI: {pct_lower:.1f}% to {pct_upper:.1f}%)")
print(f"\nStatistical significance: p = {att_results['p_value']:.4f}")
if att_results['p_value'] < 0.05:
    print("Result is statistically significant at the 5% level.")
print("="*60)

INTERPRETATION

The PLH reform is estimated to increase first-timer allocation
probability in prime locations by approximately 64.7%
(95% CI: 20.9% to 175.1%)

Statistical significance: p = 0.0292
Result is statistically significant at the 5% level.


In [14]:
# =============================================================================
# SAVE OBJECTS FOR PART 2
# =============================================================================

import pickle

# Save model and data for Part 2
objects_to_save = {
    'cfdid': cfdid,
    'analysis_df_clean': analysis_df_clean,
    'ANALYSIS_FEATURES': ANALYSIS_FEATURES,
    'TREATMENT_PERIOD': TREATMENT_PERIOD,
    'TREATMENT_DATE': TREATMENT_DATE,
    'CB_COLORS': CB_COLORS,
    'att_results': att_results
}

with open(OUTPUT / 'part1_objects.pkl', 'wb') as f:
    pickle.dump(objects_to_save, f)

print("Objects saved for Part 2 notebook.")
print(f"Output file: {OUTPUT / 'part1_objects.pkl'}")

Objects saved for Part 2 notebook.
Output file: /content/drive/MyDrive/thesis/FINAL_OUTPUT/part1_objects.pkl


---
## Summary & Next Steps

### Key Results from Part 1

1. **Data**: 493 observations spanning Nov 2011 - Aug 2023, with 23 treated post-reform observations
2. **Feature Selection**: VIF analysis reduced features from 23 to 21, removing `pc_gdp` and `cpi` due to high multicollinearity
3. **Main Estimate**: ATT ≈ 0.50 (SE ≈ 0.23), corresponding to ~50% increase in first-timer allocation probability

### Proceed to Part 2

Part 2 contains:
- Parallel trends tests (naive and conditional)
- Event study analysis
- Placebo tests
- Anticipation effects test
- Publication-ready figures
- LaTeX tables

---

**References:**

1. Gavrilova, E., Langørgen, A., & Zoutman, F. T. (2025). "Difference-in-Difference Causal Forests With an Application to Payroll Tax Incidence in Norway." *Journal of Applied Econometrics*.

2. Rambachan, A., & Roth, J. (2023). "A More Credible Approach to Parallel Trends." *Review of Economic Studies*, 90(5), 2555-2591.

3. Athey, S., & Imbens, G. W. (2022). "Design-based analysis in difference-in-differences settings with staggered adoption." *Journal of Econometrics*, 226(1), 62-79.